<h1 style="text-align: center">
Deep Learning HW4 </br>
Question 3
</h1>


### Full Name: Amirabbas Afzali

### Student ID: 400100662

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import random
import seaborn as sns
from tqdm import tqdm
from torch.utils.data import DataLoader,Dataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
import os
import pickle
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead, GPT2LMHeadModel, GPT2Config

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Dataset preparation

In [ ]:
poems = []
file = open("/content/drive/My Drive/DEEP/HW4/Q3/ferdousi.txt", "r")

while True:

  line = file.readline()
  if not line:
    break
  poems.append(line[:-1])

file.close()

poems = np.array(poems[2:])

In [ ]:
poems[:5]

array(['به نام خداوند جان و خرد', 'کزین برتر اندیشه برنگذرد',
       'خداوند نام و خداوند جای', 'خداوند روزی ده رهنمای',
       'خداوند کیوان و گردان سپهر'], dtype='<U34')

In [ ]:
inputs = poems[np.arange(0,len(poems),2)]
targets = poems[np.arange(1,len(poems),2)]

In [ ]:
inputs[:3]

array(['به نام خداوند جان و خرد', 'خداوند نام و خداوند جای',
       'خداوند کیوان و گردان سپهر'], dtype='<U34')

In [ ]:
targets[:3]

array(['کزین برتر اندیشه برنگذرد', 'خداوند روزی ده رهنمای',
       'فروزنده ماه و ناهید و مهر'], dtype='<U34')

## Load the pre-trained GPT-2 model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HooshvareLab/gpt2-fa"
ferdosi = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

----

add special tokens to tokenaizer :

In [ ]:
model_name_or_path = "HooshvareLab/gpt2-fa"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    bos_token='<s>',
    eos_token='</s>',
    pad_token='<pad>',
    unk_token='<unk>',
    # padding_size="left"
)

tokenizer.add_special_tokens({
    "bos_token": '<s>',
    "eos_token": '</s>',
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

config = AutoConfig.from_pretrained(
    model_name_or_path,
    bos_token_id=tokenizer("<s>")["input_ids"][0],
    eos_token_id=tokenizer("</s>")["input_ids"][0],
    pad_token_id=tokenizer("<pad>")["input_ids"][0],
    unk_token_id=tokenizer("<unk>")["input_ids"][0],
)


tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

In [ ]:
class Shahnameh_Dataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, max_length=128):

        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        self.attn_masks = []
        for index in range(len(inputs)-1):

            encoded_input = tokenizer('<s>' + inputs[index] + '</s>',truncation=True, max_length=max_length,padding="max_length")
            encoded_target = tokenizer('<s>' + targets[index] + '</s>',truncation=True,max_length=max_length,padding="max_length")

            self.input_ids.append(torch.tensor(encoded_input['input_ids']))
            self.target_ids.append(torch.tensor(encoded_target['input_ids']))
            mask = torch.tensor(encoded_input['attention_mask'])

            self.attn_masks.append(mask)

    def __getitem__(self, idx):
        return self.input_ids[idx],self.target_ids[idx], self.attn_masks[idx]

    def __len__(self):
        return len(self.input_ids)


Split data to train and validation:

In [ ]:
from torch.utils.data import random_split

dataset = Shahnameh_Dataset(inputs, targets, tokenizer, max_length=20)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f'Length of train dataset : {len(train_dataset)}')
print(f'Length of validation dataset : {len(val_dataset)}')

Length of train dataset : 44647
Length of validation dataset : 4961


In [ ]:
def print_poems(poem):
  print(poem.split('</s>')[0][3:])

In [ ]:
print(train_dataset[0][0].shape,'\n\n')
print_poems(tokenizer.decode(train_dataset[0][0]))
print_poems(tokenizer.decode(train_dataset[0][1]))

torch.Size([20]) 


فزونی مجوی ار شدی بی نیاز
که درد آردت پیش رنج دراز


In [ ]:
def print_poems(inpt,outpt):
  length = len(inpt)
  print('Example input:')
  print(outpt[:length])
  print('Example output:')
  print(outpt[:length],'  ,  ',outpt[length:])

In [ ]:
print_poems('تو آن خواسته گرد کن هرچ هست','تو آن خواسته گرد کن هرچ هستبست خاقوریتی دست دست')

Example input:
تو آن خواسته گرد کن هرچ هست
Example output:
تو آن خواسته گرد کن هرچ هست   ,   بست خاقوریتی دست دست


# define Dataloader

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True
)

validation_dataloader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=True
)


Our model architecture :

In [ ]:
ferdosi

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(42001, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=42001, bias=False)
)

In [ ]:
epochs = 15
warmup_steps = 1e2
sample_every = 1000

In [ ]:
optimizer = AdamW(
    ferdosi.parameters(),
    lr=5e-4,
    eps=1e-8
)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=len(train_dataloader) * epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training and Validation loop :

In [ ]:
training_stats = []
for epoch_i in range(epochs):
    print(f'Beginning epoch {epoch_i + 1} of {epochs}')

    total_train_loss = 0
    ferdosi.train()
    for step, (input,target,mask) in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):
        b_masks = torch.ones_like(b_input_ids).to(device)
        ferdosi.zero_grad()

        outputs = ferdosi(input.to(device), labels=target.to(device), attention_mask=b_masks, token_type_ids=None)

        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every 'sample_every' batches.
        if step % sample_every == 0 and not step == 0:

            print()
            print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}.')

            ferdosi.eval()

            sample_poet = inputs[np.random.randint(0, len(inputs))]
            sample_input = f"<s>{sample_poet}</s>"
            sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
            sample_input_ids = sample_input_ids.to(device)

            sample_outputs = ferdosi.generate(
                input_ids=sample_input_ids.to(device),
                attention_mask=torch.ones_like(sample_input_ids).to(device),
                # do_sample=True,
                top_k=50,
                max_length=25,
                # top_p=0.95,
                num_beams=5,
                no_repeat_ngram_size=2,
                num_return_sequences=1
            )

            for i, sample_output in enumerate(sample_outputs):
                gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
                gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
                gen_sample_output = gen_sample_output.replace("<s>", "")
                gen_sample_output = gen_sample_output.replace("</s>", "")
                gen_sample_output = gen_sample_output.replace("<sep>", "\n")
                gen_sample_output = gen_sample_output.replace("<pad>", "")
                gen_sample_output = gen_sample_output.replace("\u200c", "")
                gen_sample_output = gen_sample_output.replace("«", "") 
                gen_sample_output = gen_sample_output.replace("»", "") 
 
                print('\n')
                print_poems(sample_poet,gen_sample_output)
                print('\n')

            ferdosi.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    print()
    print(f'Epoch: {epoch_i+1}, Training Loss: {avg_train_loss}.')
    print()
    ferdosi.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for input,target,mask in tqdm(validation_dataloader, total=len(validation_dataloader), position=0):

        b_input_ids = input.to(device)
        b_labels = target.to(device)
        b_masks = torch.ones_like(b_input_ids).to(device)

        outputs = ferdosi(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)

        with torch.no_grad():
            outputs = ferdosi(b_input_ids, attention_mask=b_masks, labels=b_labels)
            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    print()
    print(f'Epoch: {epoch_i+1}, Validation Loss: {avg_val_loss}.')
    print()

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
        }
    )


Beginning epoch 1 of 15


 18%|█▊        | 999/5581 [01:24<06:17, 12.14it/s]Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.



Batch 1000 of 5581. Loss:2.322178602218628.


 18%|█▊        | 1003/5581 [01:24<08:17,  9.20it/s]



Example input:
نویسنده را پیش بنشاندند
Example output:
نویسنده را پیش بنشاندند   ,   اندنداندندند روم آمدنداندند خواندنداندند زمین




...



Example input:
برآمد چنان خسته زان آبگیر
Example output:
برآمد چنان خسته زان آبگیر   ,    تیر بود هست هستسارزد بود بود




100%|██████████| 5581/5581 [07:43<00:00, 12.04it/s]



Epoch: 15, Training Loss: 1.1958876014652913.



100%|██████████| 621/621 [00:22<00:00, 27.62it/s]


Epoch: 15, Validation Loss: 3.7166837304110687.



Continue training:

In [ ]:
epochs = 20
training_stats = []
for epoch_i in range(epochs):
    print(f'Beginning epoch {epoch_i + 1} of {epochs}')

    total_train_loss = 0
    ferdosi.train()
    for step, (input,target,mask) in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):
        b_input_ids = input.to(device)
        b_labels = target.to(device)
        b_masks = torch.ones_like(b_input_ids).to(device)

        ferdosi.zero_grad()

        outputs = ferdosi(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)

        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every 'sample_every' batches.
        if step % sample_every == 0 and not step == 0:

            print()
            print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}.')

            ferdosi.eval()

            sample_poet = inputs[np.random.randint(0, len(inputs))]
            sample_input = f"<s>{sample_poet}</s>"
            sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
            sample_input_ids = sample_input_ids.to(device)

            sample_outputs = ferdosi.generate(
                input_ids=sample_input_ids.to(device),
                attention_mask=torch.ones_like(sample_input_ids).to(device),
                # do_sample=True,
                top_k=50,
                max_length=25,
                # top_p=0.95,
                num_beams=5,
                no_repeat_ngram_size=2,
                num_return_sequences=1
            )

            for i, sample_output in enumerate(sample_outputs):
                gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
                gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
                gen_sample_output = gen_sample_output.replace("<s>", "")
                gen_sample_output = gen_sample_output.replace("</s>", "")
                gen_sample_output = gen_sample_output.replace("<sep>", "\n")
                gen_sample_output = gen_sample_output.replace("<pad>", "")
                gen_sample_output = gen_sample_output.replace("\u200c", "")
                gen_sample_output = gen_sample_output.replace("«", "")
                gen_sample_output = gen_sample_output.replace("»", "")

                print('\n')
                print_poems(sample_poet,gen_sample_output)
                print('\n')

            ferdosi.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    print()
    print(f'Epoch: {epoch_i+1}, Training Loss: {avg_train_loss}.')
    print()

    ferdosi.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for input,target,mask in tqdm(validation_dataloader, total=len(validation_dataloader), position=0):

        b_input_ids = input.to(device)
        b_labels = target.to(device)
        b_masks = torch.ones_like(b_input_ids).to(device)

        outputs = ferdosi(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)

        with torch.no_grad():

            outputs = ferdosi(b_input_ids, attention_mask=b_masks, labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    print()
    print(f'Epoch: {epoch_i+1}, Validation Loss: {avg_val_loss}.')
    print()

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
        }
    )


Beginning epoch 1 of 20


 18%|█▊        | 1000/5581 [01:23<06:25, 11.88it/s]Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.



Batch 1000 of 5581. Loss:2.761352300643921.


 18%|█▊        | 1002/5581 [01:24<14:14,  5.36it/s]



Example input:
همین داشتم چشم زان بد نهان
Example output:
همین داشتم چشم زان بد نهان   ,   ویدیودیودیوودیوویدیو وو




...



Example input:
یکی نامه با هدیه شاهوار
Example output:
یکی نامه با هدیه شاهوار   ,    نگارار و باز کرد به آن بر بیار




100%|██████████| 5581/5581 [07:44<00:00, 12.00it/s]



Epoch: 20, Training Loss: 1.1951289343227305.



100%|██████████| 621/621 [00:22<00:00, 28.21it/s]


Epoch: 20, Validation Loss: 3.6084791826933094.



# Inference :

In [ ]:
def random_generator():
  ferdosi.eval()

  sample_poet = inputs[np.random.randint(0, len(inputs))]
  sample_input = f"<s>{sample_poet}</s>"
  sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
  sample_input_ids = sample_input_ids.to(device)

  sample_outputs = ferdosi.generate(
      input_ids=sample_input_ids.to(device),
      attention_mask=torch.ones_like(sample_input_ids).to(device),
      top_k=50,
      # top_p=0.50,
      max_length=20,
      num_beams=5,
      no_repeat_ngram_size=1,
      num_return_sequences=1,
      do_sample=False,
      # pad_token_id=tokenizer.eos_token_id,
  )

  for i, sample_output in enumerate(sample_outputs):
      gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
      gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
      gen_sample_output = gen_sample_output.replace("<s>", "")
      gen_sample_output = gen_sample_output.replace("</s>", "")
      gen_sample_output = gen_sample_output.replace("<sep>", "\n")
      gen_sample_output = gen_sample_output.replace("<pad>", "")
      gen_sample_output = gen_sample_output.replace("\u200c", " ")
      gen_sample_output = gen_sample_output.replace("«", " ")
      gen_sample_output = gen_sample_output.replace("»", " ")

      print_poems(sample_poet,gen_sample_output)



In [ ]:
for _ in range(10):
  random_generator()
  print('\n\n')

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


Example input:
به درگاه چون خواست لشکر فزون
Example output:
به درگاه چون خواست لشکر فزون   ,    و چندب نی آب ز آرم سان گذشت گود



Example input:
چه نیکو بود هر که خود را شناخت
Example output:
چه نیکو بود هر که خود را شناخت   ,   ود چند دخترش دیگر فریبرز بهست سرود سو





Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


Example input:
که گر گیو و خسرو به ایران شوند
Example output:
که گر گیو و خسرو به ایران شوند   ,    شکار زوار عیسی شاهکوی بدتری دهی



Example input:
بدین جستن عیبهای دروغ
Example output:
بدین جستن عیبهای دروغ   ,    فروغستی دخترش شد آزمود ایچ فرستیمگرفترد





Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


Example input:
دگر کز پی جنگ افراسیاب
Example output:
دگر کز پی جنگ افراسیاب   ,    و خواب آبز وس از پرشتاب به ز بر راه



Example input:
یکی بچهٔ فرخ آمد پدید
Example output:
یکی بچهٔ فرخ آمد پدید   ,    رسیدید دیدزیدودزدی بر برو س گستر پیران





Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


Example input:
بپوشید پس جامهٔ نو سپید
Example output:
بپوشید پس جامهٔ نو سپید   ,   ستازز شد شست گشت دل ایچ اندرید



Example input:
گزین کرد زو بارگی ده هزار
Example output:
گزین کرد زو بارگی ده هزار   ,    سوار شکارارسار نشانخر کشوری ز شمارزار





Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


Example input:
چنین گفت زان پس به بانگ بلند
Example output:
چنین گفت زان پس به بانگ بلند   ,   ندود چند دودوبندلقست سرود راز باز



Example input:
چنین گفت ماهوی بدبخت و شوم
Example output:
چنین گفت ماهوی بدبخت و شوم   ,    بوم رومز چند دخترشست برو شویم دراز نیاز





## Save `ferdosi` :)

In [ ]:
torch.save(ferdosi.state_dict(), '/content/drive/My Drive/DEEP/HW4/Q3/ferdosi.pth')

## Report :


<div dir="rtl" style="font-size:18px">
در این تمرین از fine tune کردن gpt2 برای تولید اشعار فردوسی استفاده شده است. و خروجی ی مدل به ازای 40 ایپاک در حالت inference در چند مثال بالا آمده است.

همچنین خروجی مدل که در هر ایپاک به عنوان training loss نمایش داده شده است ، همان perplexity می باشد.
</div>

# END :\